In [106]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

In [107]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.5


In [108]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [109]:
//////////////////////////////////////////////////////////////////

val Model = PipelineModel.load(modelPath)

val predictionsDF = Model.transform(inputDF)


Model = pipeline_ebfe683b7a32
predictionsDF = [tweet: string, words: array<string> ... 4 more fields]


[tweet: string, words: array<string> ... 4 more fields]

In [110]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}.start()

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|is casting away c...|[is, casting, awa...|(1000,[281,416,73...|[4.87917110966780...|[0.48791711096678...|       1.0|
|@YannR 15 miles 1...|[@yannr, 15, mile...|(1000,[170,207,22...|[5.07035085442434...|[0.50703508544243...|       0.0|
|@Chesska5 AGAIN!!...|[@chesska5, again...|(1000,[372,412,44...|[4.87917110966780...|[0.48791711096678...|       1.0|
|@vodafoneNZ why n...|[@vodafonenz, why...|(1000,[7,40,224,2...|[5.71261516780369...|[0.57126151678036...|       0.0|
|mark derosa AND k...|[mark, derosa, an...|(1000,[47,82,109,...|[5.63794180065782...|[0.56379418006578...|       0.0|
|@toeandno glad to...|[@toeandno, glad,...|(1000,[73,77,

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3ef1bcf1

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|caramel was a bad...|[caramel, was, a,...|(1000,[86,170,234...|[5.14697803216483...|[0.51469780321648...|       0.0|
|@kim_chi oh im lo...|[@kim_chi, oh, im...|(1000,[32,157,159...|[5.60333639419692...|[0.56033363941969...|       0.0|
|@KILA21  nov 20th...|[@kila21, , nov, ...|(1000,[7,368,372,...|[5.18576025215998...|[0.51857602521599...|       0.0|
|@chelseaheyy, ahh...|[@chelseaheyy,, a...|(1000,[34,54,72,9...|[4.94764312568262...|[0.49476431256826...|       1.0|
|gettin a tat haha...|[gettin, a, tat, ...|(1000,[7,17,74,17...|[4.84456570320690...|[0.48445657032069...|       1.0|
|@MGiraudOfficial ...|[@mgiraudofficial...|(1000,[0,188,

In [111]:
import org.apache.spark.ml.linalg.DenseVector
val toArr: Any => Array[Double] = _.asInstanceOf[DenseVector].toArray
val toArrUdf = udf(toArr)

predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF
        .withColumn("probability_array", toArrUdf('probability))
        .withColumn("negative_probability", 'probability_array(1))
        .select($"tweet", $"negative_probability")
        .show()
}.start()


+--------------------+--------------------+
|               tweet|negative_probability|
+--------------------+--------------------+
|is casting away c...|   0.512082889033219|
|@YannR 15 miles 1...|  0.4929649145575655|
|@Chesska5 AGAIN!!...|   0.512082889033219|
|@vodafoneNZ why n...| 0.42873848321963104|
|mark derosa AND k...|   0.436205819934217|
|@toeandno glad to...|  0.5379354453350196|
|On the streetcar ...|  0.4918990234192786|
|@jenmarielle why ...|  0.5920915934117577|
|@mikeypiff you're...|  0.5338122294901468|
|@sunanna ooo! I w...|  0.5091076203476022|
|today was the bes...|  0.5688520600607138|
|  new background... |   0.512082889033219|
|@kyle270 hey baby...|   0.578296117697853|
|@realmfox You loo...|  0.5437651845545968|
|      ciao for now! |   0.512082889033219|
|Mmmm frozen yogurt. |   0.512082889033219|
|@KimKardashian ye...|   0.549347621510118|
|@SinaAmedson it's...|  0.5437651845545968|
|     Goodmorning !! |   0.512082889033219|
|@FitPole Very tem...|  0.525219

toArr = > Array[Double] = <function1>
toArrUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


lastException: Throwable = null


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@590c926c